# Advanced Prompting Patterns: ReAct (Reasoning + Acting)

This notebook demonstrates the ReAct pattern, a powerful technique for building autonomous agents. ReAct combines **reasoning** (Chain-of-Thought) with **acting** (Tool Use).

## How it works

The model operates in a loop:
1. **Thought**: Analyze the current state and decide what to do.
2. **Action**: Select a tool and parameters.
3. **Observation**: Receive the output from the tool.
4. **Repeat**: Continue until the task is solved.

In [ ]:
import os
import re

# Placeholder for LLM call - would use OpenAI/Anthropic API in production
def call_llm(prompt):
    # In a real scenario, this would call GPT-4 or Claude
    print("\n--- SENDING TO LLM ---\n")
    print(prompt)
    print("\n----------------------\n")
    
    # Simulating a response for demonstration purposes
    # Scene: User asks "What is the weather in Tokyo and is it good for a picnic?"
    if "Observation:" not in prompt:
        return "Thought: I need to find the weather in Tokyo first.\nAction: weather(location=\"Tokyo\")"
    elif "15C, Cloudy" in prompt:
        return "Thought: The weather is cloudy and 15C. That's a bit cool. I should check if it's raining.\nAction: weather_details(location=\"Tokyo\")"
    elif "Chance of rain: 40%" in prompt:
        return "Thought: There is a 40% chance of rain. That makes it risky for a picnic.\nAnswer: It is 15C and cloudy in Tokyo with a 40% chance of rain. It might not be the best day for a picnic."
    return "Error: Unexpected state"

## 1. Define Tools

Tools are functions the agent can call. In this simple example, we mock a weather tool.

In [ ]:
def weather_tool(location):
    if "Tokyo" in location:
        return "15C, Cloudy"
    return "Unknown location"

def weather_details_tool(location):
    if "Tokyo" in location:
        return "Chance of rain: 40%"
    return "No details available"

tools = {
    "weather": weather_tool,
    "weather_details": weather_details_tool
}

## 2. ReAct Prompt Template

The prompt instructs the model on how to interleave thoughts and actions.

In [ ]:
react_template = """
You are an assistant with access to the following tools:

- weather(location): Get the current temperature and condition.
- weather_details(location): Get precipitation/wind details.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [weather, weather_details]
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Answer: the final answer to the original input question

Begin!

Question: {question}
"""

## 3. The Agent Loop

This logic parses the model output, executes tools, and feeds the result back into the prompt.

In [ ]:
def run_react_agent(question):
    prompt = react_template.format(question=question)
    
    for i in range(5): # Max steps
        response = call_llm(prompt)
        print(f"[Step {i+1} Model Output]: {response}")
        
        # Check if we have an answer
        if "Answer:" in response:
            return response.split("Answer:")[1].strip()
        
        # Parse Action
        action_match = re.search(r"Action: (\w+)\((.*?)\)", response)
        if action_match:
            tool_name = action_match.group(1)
            tool_arg = action_match.group(2).replace('"', '')
            
            print(f"[Executing Tool]: {tool_name} with arg {tool_arg}")
            
            if tool_name in tools:
                observation = tools[tool_name](tool_arg)
            else:
                observation = "Error: Tool not found"
            
            print(f"[Observation]: {observation}")
            
            # Append to history
            prompt += response + "\nObservation: " + observation + "\n"
        else:
            print("Failed to parse action")
            break

In [ ]:
run_react_agent("What is the weather in Tokyo and is it good for a picnic?")